# `TohuNamespaceNEW2`


[TOC]


The class `TohuNamespaceNEW2` allows grouping together other tohu generators and treating them as a single unit (which is used to implement the functionality of custom generators).

## Initialisation and adding field generators

In [21]:
from tohu import Integer, HashDigest, FakerGenerator
from tohu.tohu_namespace_NEW_2 import TohuNamespaceNEW2

In [4]:
g1 = Integer(100, 200)
g2 = HashDigest(length=6)
g3 = FakerGenerator(method="name")

tohu_namespace = TohuNamespaceNEW2()
tohu_namespace.add_field_generator("aa", g1)
tohu_namespace.add_field_generator("bb", g2)
tohu_namespace.add_field_generator("cc", g3)

Note that when a field generator is added to the tohu namesapce, internally a new spawn is created. We can verify this by checking that the generators are different, for example in the case of `g1` and the field generator `aa`:

In [5]:
print(f"g1: {g1}")
print(f"aa: {tohu_namespace.field_generators['aa']}")

assert tohu_namespace.field_generators['aa'] is not g1
assert not tohu_namespace.field_generators['aa'].is_clone_of(g1)

g1: <Integer (id=c30ab4)>
aa: <Integer (id=a118b1)>


There is a convenience method `add_field_generators_from_dict()` which allows passing a dictionary, and it will call `add_field_generator()` for any tohu generators found in this dictionary (while ignoring any other values).

In [6]:
dct = {
    "aa": Integer(100, 200),
    "bb": HashDigest(length=6),
    "some_string": "this string will not be added because it is not a tohu generator",
    "cc": FakerGenerator(method="name"),
    "answer": 42  # this number will also not be added because it is not a tohu generator
}

tohu_namespace = TohuNamespaceNEW2()
tohu_namespace.add_field_generators_from_dict(dct)

In [7]:
tohu_namespace.field_generators

{'aa': <Integer (id=a0902f)>,
 'bb': <HashDigest (id=b6f737)>,
 'cc': <FakerGenerator (id=2d9a7c)>}

## Adding non-field generators

In [8]:
from tohu.looping_NEW import LoopVariableNEW

In [9]:
xx = LoopVariableNEW("xx", values=[4, 5, 6])

tohu_namespace = TohuNamespaceNEW2()
tohu_namespace.add_non_field_generator("xx", xx, is_externally_managed=True)

In [10]:
tohu_namespace.field_generators

{}

In [11]:
tohu_namespace.all_generators

{'xx': <LoopVariable: name='xx', loop_level=None, values=[4, 5, 6], cur_value=4 (tohu_id=fd44cd)>}

Note that in this case the generator which is internally stored in the tohu namespace is actually a clone of `xx`.

In [12]:
assert tohu_namespace.all_generators["xx"].is_clone_of(xx)

## Setting the `tohu_items_cls` attribute

Let's create a new tohu namespace and add both field generators and non-field generators.

In [13]:
xx = LoopVariableNEW("xx", values=[111, 222]).set_loop_level(2)
yy = LoopVariableNEW("yy", values=["AAA", "BBB"]).set_loop_level(1)

g1 = Integer(100, 200)
g2 = HashDigest(length=6)
g3 = FakerGenerator(method="name")

tohu_namespace = TohuNamespaceNEW2()
tohu_namespace.add_non_field_generator("xx", xx, is_externally_managed=True)
tohu_namespace.add_non_field_generator("yy", yy, is_externally_managed=True)
tohu_namespace.add_field_generator("aa", g1)
tohu_namespace.add_field_generator("bb", g2)
tohu_namespace.add_field_generator("cc", g3)

Initially the `tohu_items_cls` attribute refers to a non-existent tohu items class:

In [14]:
tohu_namespace.tohu_items_cls

<NonExistentTohuItemsClass>

Once all desired generators have been added to the tohu namespace, we can call `set_tohu_items_class`, which will automatically create a tohu items class with the same field names as the generators contained in the namespace.

In [15]:
tohu_namespace.set_tohu_items_class(name="Quux")

In [16]:
tohu_namespace.tohu_items_cls

tohu.tohu_items_class.Quux

This items class can then be used to create individual tohu items.

In [17]:
tohu_namespace.tohu_items_cls(aa=100, bb="910A97", cc="Kristen Wallace")

Quux(aa=100, bb='910A97', cc='Kristen Wallace')

## Resetting and generating tohu items

In [18]:
tohu_namespace.all_generators

{'xx': <LoopVariable: name='xx', loop_level=2, values=[111, 222], cur_value=111 (tohu_id=1645bf)>,
 'yy': <LoopVariable: name='yy', loop_level=1, values=['AAA', 'BBB'], cur_value='AAA' (tohu_id=789d29)>,
 'aa': <Integer (id=4dbbbb)>,
 'bb': <HashDigest (id=dd4f62)>,
 'cc': <FakerGenerator (id=d0d2a7)>}

In [19]:
tohu_namespace.field_generators

{'aa': <Integer (id=4dbbbb)>,
 'bb': <HashDigest (id=dd4f62)>,
 'cc': <FakerGenerator (id=d0d2a7)>}

In [20]:
tohu_namespace.reset(seed=11111)

print(next(tohu_namespace))
print(next(tohu_namespace))
print(next(tohu_namespace))
print(next(tohu_namespace))
print(next(tohu_namespace))

Quux(aa=163, bb='7551AA', cc='Michelle Miller')
Quux(aa=171, bb='54596E', cc='Eddie Davis')
Quux(aa=142, bb='2A16D0', cc='Kathleen Lucas')
Quux(aa=140, bb='FDCDD3', cc='Jason Rodriguez')
Quux(aa=121, bb='BDE283', cc='Andrew Pitts')


**TODO:**

- [ ] Add test that field generators are reset when the tohu namespace is reset, but externally managed (non-field?) generators are left alone.